In [1]:
import torch
from torch import nn
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torchvision import datasets
import matplotlib.pyplot as plt

In [2]:
train_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

batch_size =64

train_dataloader = DataLoader(train_data , batch_size = batch_size)
test_dataloader = DataLoader(test_data , batch_size = batch_size)

print(f"train shape before loading : {len(train_data)}")
print(f"test shape before loading : {len(train_data)}")
print(f"train shape after loading : {len(train_dataloader)*batch_size}")
print(f"test shape after loading : {len(test_dataloader)*batch_size}")

train shape before loading : 60000
test shape before loading : 60000
train shape after loading : 60032
test shape after loading : 10048


#setting up the Neural Network

In [3]:
device = None

if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
print(f"using {device}")

device = torch.device(device)

using cpu


In [4]:
class NeuralNetworks(nn.Module):
  def __init__(self ,input_size , hidden_layer , output_size):
    super().__init__()
    self.flatten = nn.Flatten()
    self.Linear_relu_stack = nn.Sequential(
        nn.Linear(input_size , hidden_layer),
        nn.ReLU(),
        nn.Linear(hidden_layer , hidden_layer),
        nn.ReLU(),
        nn.Linear(hidden_layer , output_size)
    )

  def forward(self,image_tensor):
    image_tensor = self.flatten(image_tensor)
    pred = self.Linear_relu_stack(image_tensor)
    return pred

In [5]:
input_size = 28*28
hidden_layer = 512
output_size = 10
model = NeuralNetworks(input_size , hidden_layer , output_size).to(device)
print(model)

NeuralNetworks(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (Linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [6]:
lr = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)


In [13]:
def train(DataLoader):
  size = len(DataLoader.dataset)
  model.train()

  for batch_num , (X,y) in enumerate(DataLoader):
    x , y = X.to(device) , y.to(device)

    pred = model(x)
    loss = loss_fn(pred,y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_num>0 and batch_num%100 ==0:
      loss = loss.item()
      print(f"loss : {loss:2f}")

In [16]:
def test(DataLoader):
  size = len(DataLoader.dataset)
  batch_size = (DataLoader)

  model.eval()
  loss , correct = 0,0
  for batch_num , (X,y) in enumerate(DataLoader):
    x ,y = X.to(device) , y.to(device)
    pred = model(x)
    loss += loss_fn(pred,y)
    correct += (pred.argmax(1)==y).type(torch.float).sum().item()

  loss /= size
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")


In [17]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader)
    test(test_dataloader)
print("Done!")

Epoch 1
-------------------------------
loss : 0.371225
loss : 0.300885
loss : 0.376256
loss : 0.397656
loss : 0.354059
loss : 0.332847
loss : 0.467751
loss : 0.381924
loss : 0.475219
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.006215 

Epoch 2
-------------------------------
loss : 0.340598
loss : 0.241491
loss : 0.338096
loss : 0.352208
loss : 0.338661
loss : 0.287123
loss : 0.403545
loss : 0.345174
loss : 0.401925
Test Error: 
 Accuracy: 86.2%, Avg loss: 0.005904 

Epoch 3
-------------------------------
loss : 0.319887
loss : 0.231184
loss : 0.258282
loss : 0.337457
loss : 0.306257
loss : 0.266266
loss : 0.353758
loss : 0.286714
loss : 0.318207
Test Error: 
 Accuracy: 86.3%, Avg loss: 0.005838 

Epoch 4
-------------------------------
loss : 0.279443
loss : 0.216528
loss : 0.230248
loss : 0.346719
loss : 0.262817
loss : 0.228715
loss : 0.338294
loss : 0.290930
loss : 0.277929
Test Error: 
 Accuracy: 86.4%, Avg loss: 0.005825 

Epoch 5
-------------------------------
loss : 0.222154


In [25]:
torch.save(model.state_dict(),"myfirstnn.pth")
print("saved pytorch model state to myfirstnn")

model = NeuralNetworks(input_size , hidden_layer , output_size)
model.load_state_dict(torch.load("myfirstnn.pth"))

model.eval()

for i in range(10):
  x,y = test_data[i][0],test_data[i][1]
  x = x.to(device)
  pred = model(x)
  pred , actual = pred.argmax(1) , y
  print(f" predicted : {pred} actual : {actual}")

saved pytorch model state to myfirstnn
 predicted : tensor([9]) actual : 9
 predicted : tensor([2]) actual : 2
 predicted : tensor([1]) actual : 1
 predicted : tensor([1]) actual : 1
 predicted : tensor([6]) actual : 6
 predicted : tensor([1]) actual : 1
 predicted : tensor([4]) actual : 4
 predicted : tensor([6]) actual : 6
 predicted : tensor([5]) actual : 5
 predicted : tensor([7]) actual : 7


<ipython-input-25-be6e0e2b9ac1>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("myfirstnn.pth"))
